<html>
<head>
	<title></title>
</head>
<body>
<h1 style="text-align: center;line-height: 120%;">ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ВЫСШЕГО ОБРАЗОВАНИЯ НАЦИОНАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ УНИВЕРСИТЕТ «МОСКОВСКИЙ ЭНЕРГЕТИЧЕСКИЙ ИНСТИТУТ» Кафедра «РЗиАЭ»</h1>
<h2 style="text-align: center;line-height: 20%;">Лабораторная работа №1</h2>

<h3 style="text-align: center;line-height: 80%;"><em>"Исследование муравьиного алгоритма для решения задачи коммивояжера"</em></h3>

<p style="text-align: right;line-height: 40%;">Выполнил: студент группы Э-13м-19</p>

<p style="text-align: right;line-height: 40%;">Павлов И. А.</p>

<p style="text-align: right;line-height: 40%;">Принял: Грачева Н. П.</p>
<h4 style="text-align: center;line-height: 100%;"><em> Москва, 2020</em></h3>
</body>
</html>

### Импорт библиотек необходимых для работы

In [2]:
import numpy as np
import random
import time

### Исходые данные

In [10]:
data = np.load('./Task9.npy')
print("Матрица путей: \n" + str(data))

Матрица путей: 
[[  0   9  77  95  89  74  33  32 100  60]
 [  9   0  74   4  82  85   6  14  36   3]
 [ 77  74   0  92 100  71  95  49  92  77]
 [ 95   4  92   0  12  80  87   4  99  65]
 [ 89  82 100  12   0  31  67  70  34  29]
 [ 74  85  71  80  31   0  46  46  75  79]
 [ 33   6  95  87  67  46   0  15   2  66]
 [ 32  14  49   4  70  46  15   0  60  56]
 [100  36  92  99  34  75   2  60   0  73]
 [ 60   3  77  65  29  79  66  56  73   0]]


### Реализация муравьиного алгоритма

In [11]:
def antWay(numIterat, numAnts):
    # Формирование матрицы, состоящей из единиц, для феромонов
    phermony = np.ones((10, 10))
    WinPath = []
    WinLenght = 10000
    # Запуск таймера
    StartTime = time.process_time()
    # Итерация
    for n in range(0, numIterat):
        # Движение популяции
        for a in range(0, numAnts):
            # Формирование необходимых переменных для расчета, массив городов, лист пути, начальная точка, длина пути
            cityes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
            Way = []
            Punkt = 1
            LenghtWay = 0
            cityes.remove(Punkt)
            Way.append(Punkt)
            # Пока не пройдет через все точки, продолжает идти
            while len(cityes) != 0:
                # Массив вероятностей для всех городов
                p = []
                for i in cityes:
                    p.append(pForNextPunkt(cityes, Punkt, phermony, data, i))
                # Запускаем рулетку для определения следующего города
                NextPunkt = Roulette(cityes, p)
                # Формирование длинны пути
                LenghtWay += data[Punkt-1][NextPunkt-1]
                # Определяем новую стартовую точку
                Punkt = NextPunkt
                # Заполняем матрицу пути
                Way.append(Punkt)
                cityes.remove(Punkt)
            # Обновление феромонов на ребрах
            for i in range(0, len(Way)):
                if i == len(Way) - 1: break
                addPheromony(phermony, Way[i], Way[i + 1], LenghtWay)
            # Поиск оптимального пути у популяции
            if LenghtWay <= WinLenght:
                WinLenght = LenghtWay
                WinPath = Way
        # Досрочный выход, при достижении заданной точности
        if WinLenght <=221: break

    # Вывод результатов
    EndTime = time.process_time()
    Time = EndTime - StartTime
    print("Way: " + str(WinPath))
    print("Way length: " + str(WinLenght))
    print("Time: " + str(Time))
    
    # Функция нахождения  вероятности перехода
def pForNextPunkt(cityes, currentPunkt, phermony, data, NextPunkt):
    # Коэффициенты альфа и бетта
    a = 1
    b = 1
    denominator = 0
    # Сдвиг для работы с матрицей
    currentPunkt -= 1
    NextPunkt -= 1
    # Получаем знаменатель, для дальнейшего поиска вероятности
    for i in cityes:
        denominator += (phermony[currentPunkt][i-1] ** a) / (data[currentPunkt][i-1] ** b)
    # Находим вероятность
    pForNextPunkt = (phermony[currentPunkt][NextPunkt] ** a) / (data[currentPunkt][NextPunkt] ** b) / denominator
    return pForNextPunkt

# Рулетка, для выбора следующего города
def Roulette(cities, P):
    # Рандомный выстрел
    randValue = random.random()
    summ = 0
    IndexNextPoint = 0
    # Суммируем, пока наше знаение выстрела не станет меньше, чем сумма
    for i in P:
        summ += i
        if randValue < summ:
            # Фиксируем зону в которую попал выстрел
            IndexNextPoint = P.index(i)
            break
    # Определяем город, в который двигаемся дальше 
    nextPoint = cities[IndexNextPoint]
    return nextPoint

# Финкция изменения матрицы феромонов
def addPheromony(pheromony, startPunkt, endPunkt, Lenght):
    # Задаем значения феромонов на весь путь и коэффициент рассеивания
    Q = 100
    ro = 0.9
    pheromony[startPunkt - 1][endPunkt - 1] = ro * pheromony[startPunkt - 1][endPunkt - 1] + (Q / Lenght)

# Запуск алгоритма
Ants = 5
antWay(100, Ants)

Way: [1, 2, 10, 5, 4, 8, 7, 9, 6, 3]
Way length: 220
Time: 0.0625


### Элитные муравьиные системы

In [14]:
def antWay(numIterat, numAnts):
    phermony = np.ones((10, 10))
    WinPath = []
    WinLenght = 10000
    StartTime = time.process_time()
    # Iterate
    for n in range(0, numIterat):
        for a in range(0, numAnts):
            cityes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
            Way = []
            Punkt = 1
            LenghtWay = 0
            cityes.remove(Punkt)
            Way.append(Punkt)
            while len(cityes) != 0:
                p = []
                for i in cityes:
                    p.append(pForNextPunkt(cityes, Punkt, phermony, data, i))
                NextPunkt = Roulette(cityes, p)
                LenghtWay += data[Punkt-1][NextPunkt-1]
                Punkt = NextPunkt
                Way.append(Punkt)
                cityes.remove(Punkt)
            if LenghtWay <= WinLenght:
                WinLenght = LenghtWay
                WinPath = Way

        if WinLenght <=221: break
        # Феромоны обновляются только для одного лучшего пути
        for i in range(0, len(WinPath)):
            if i == len(WinPath) - 1: break
            addPheromony(phermony, WinPath[i], WinPath[i + 1], WinLenght)
        print(str(WinPath) + "  " + str(WinLenght))
        print(phermony)

    EndTime = time.process_time()
    Time = EndTime - StartTime
    print("Way: " + str(WinPath))
    print("Way length: " + str(WinLenght))
    print("Time: " + str(Time))


def pForNextPunkt(cityes, currentPunkt, phermony, data, NextPunkt):
    a = 1
    b = 1
    denominator = 0
    currentPunkt -= 1
    NextPunkt -= 1
    for i in cityes:
        denominator += (phermony[currentPunkt][i-1] ** a) / (data[currentPunkt][i-1] ** b)
    pForNextPunkt = (phermony[currentPunkt][NextPunkt] ** a) / (data[currentPunkt][NextPunkt] ** b) / denominator
    return pForNextPunkt


def Roulette(cities, P):
    randValue = random.random()
    summ = 0
    IndexNextPoint = 0
    for i in P:
        summ += i
        if randValue < summ:
            IndexNextPoint = P.index(i)
            break
    nextPoint = cities[IndexNextPoint]
    return nextPoint


def addPheromony(pheromony, startPunkt, endPunkt, Lenght):
    Q = 500
    ro = 0.9
    pheromony[startPunkt - 1][endPunkt - 1] = ro * pheromony[startPunkt - 1][endPunkt - 1] + (Q / Lenght)
    # Ограничение феромонов сверху и снизу
    if pheromony[startPunkt - 1][endPunkt - 1] > 3:
        pheromony[startPunkt - 1][endPunkt - 1] = 3
    if pheromony[startPunkt - 1][endPunkt - 1] < 0.5:
        pheromony[startPunkt - 1][endPunkt - 1] = 0.5

Ants = 5
antWay(50, Ants)

[1, 9, 7, 2, 4, 8, 10, 3, 6, 5]  351
[[1.         1.         1.         1.         1.         1.
  1.         1.         2.32450142 1.        ]
 [1.         1.         1.         2.32450142 1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         2.32450142
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         2.32450142 1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         2.32450142 1.
  1.         1.         1.         1.        ]
 [1.         2.32450142 1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         2.32450142]
 [1.         1.         1.         1.         1.         1.
  2.32450142 1.         1.         1.  

### Алгоритмы муравьиной колонии

In [15]:
def antWay(numIterat, numAnts):
    phermony = np.ones((10, 10))
    WinPath = []
    WinLenght = 10000
    StartTime = time.process_time()
    q0 = 0.3
    for n in range(0, numIterat):
        for a in range(0, numAnts):
            # Forming a list of cities, the first city, the length of the path, the path
            cityes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
            Way = []
            Punkt = 1
            LenghtWay = 0
            cityes.remove(Punkt)
            Way.append(Punkt)
            while len(cityes) != 0:
                # Выбор пути используя псевдо - случайный пропорциональный выбор
                q = random.random()
                if q < q0:
                    NextPunkt = randNextPunkt(cityes, phermony, Punkt)
                else:
                    p = []
                    for i in cityes:
                        p.append(pForNextPunkt(cityes, Punkt, phermony, data, i))
                    NextPunkt = Roulette(cityes, p)
                LenghtWay += data[Punkt-1][NextPunkt-1]
                # Дополнительное локальное обновление, выполняемое каждым муравьем сразу же после прохода по ребру (i, j)
                AntColony(phermony, Punkt, NextPunkt)
                Punkt = NextPunkt
                Way.append(Punkt)
                cityes.remove(Punkt)
            if LenghtWay <= WinLenght:
                WinLenght = LenghtWay
                WinPath = Way

        if WinLenght <=221: break
        # Глобальное обновление феромонов
        for i in range(0, len(WinPath)):
            if i == len(WinPath) - 1: break
            addPheromony(phermony, WinPath[i], WinPath[i + 1], WinLenght)
        print(str(WinPath) + "  " + str(WinLenght))
        print(phermony)

    EndTime = time.process_time()
    Time = EndTime - StartTime
    print("Way: " + str(WinPath))
    print("Way length: " + str(WinLenght))
    print("Time: " + str(Time))


def pForNextPunkt(cityes, currentPunkt, phermony, data, NextPunkt):
    a = 1
    b = 1
    denominator = 0
    currentPunkt -= 1
    NextPunkt -= 1
    for i in cityes:
        denominator += (phermony[currentPunkt][i-1] ** a) / (data[currentPunkt][i-1] ** b)
    pForNextPunkt = (phermony[currentPunkt][NextPunkt] ** a) / (data[currentPunkt][NextPunkt] ** b) / denominator
    return pForNextPunkt


def Roulette(cities, P):
    randValue = random.random()
    summ = 0
    IndexNextPoint = 0
    for i in P:
        summ += i
        if randValue < summ:
            IndexNextPoint = P.index(i)
            break
    nextPoint = cities[IndexNextPoint]
    return nextPoint


def addPheromony(pheromony, startPunkt, endPunkt, Lenght):
    Q = 500
    ro = 0.9
    pheromony[startPunkt - 1][endPunkt - 1] = ro * pheromony[startPunkt - 1][endPunkt - 1] + (Q / Lenght)

# Локальное обновление феромонов    
def AntColony(pheromony, startPunkt, endPunkt):
    fi = 0.5
    pheromony[startPunkt - 1][endPunkt - 1] = (1 - fi) * pheromony[startPunkt - 1][endPunkt - 1] + fi * 1

# Выбор следующей точки, с наибольшим количеством феромонов    
def randNextPunkt(cityes, phermony, Punkt):
    maxValue = 0
    for i in cityes:
        if phermony[Punkt-1][i-1] > maxValue:
            maxValue = phermony[Punkt-1][i-1]
            nextPunkt = i
    return nextPunkt

Ants = 5
antWay(100, Ants)

[1, 2, 4, 8, 3, 5, 6, 9, 7, 10]  340
[[1.         2.37058824 1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         2.37058824 1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         2.37058824 1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         2.37058824 1.         1.        ]
 [1.         1.         1.         1.         1.         2.37058824
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         2.37058824 1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         2.37058824]
 [1.         1.         2.37058824 1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  2.37058824 1.         1.         1.  